# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [2]:
import pandas as pd
import googlemaps
import numpy as np
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [3]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [4]:
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Balsas


Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [5]:
# Creando las listas
provincias = list(ubigeos['PROVINCIA'])
distritos = list(ubigeos['DISTRITO'])

Limpiando las listas

In [6]:
# Eliminando duplicados
provincias = set(provincias)
distritos = set(distritos)

# Eliminando algunos elementos inutiles ('', nan)
provincias = list(filter(lambda x: str(x) != 'nan', provincias))
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

provincias.remove('')
provincias.remove('PROVINCIA')
distritos.remove('')

Eliminando las tildes

In [7]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [8]:
# La aplicamos a cada elemento de nuestras provincias/distritos
nro_provs = len(provincias)
for i in range(nro_provs):
    provincias[i] = normalize(provincias[i])

nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])

In [11]:
# print(provincias)
# print(distritos)

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia esta [guía](https://www.datahubbs.com/google-maps-python/)

### Matriz a nivel de Provincias

Lo primero será crear un DataFrame con los distritos/provincias de origen y de llegada.

In [17]:
first_col = ["Origin Province\Destination Province"]
locations = pd.DataFrame({"Origin Province\Destination Province": provincias}, 
                        columns = first_col+provincias)
locations

,Origin Province\Destination Province,Asuncion,Huanuco,Chiclayo,El Collao,Nasca,Santa,Utcubamba,Huamalies,Barranca,...,Santa Cruz,Tarata,Carhuaz,Huarochiri,Yarowilca,Ica,Huancavelica,Mariscal Caceres,Chepen,Camana
0,Asuncion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Huanuco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiclayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,El Collao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nasca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Ica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,Huancavelica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,Mariscal Caceres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Chepen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
for value in locations["Origin Province\Destination Province"]:
    print(value)
    for column in locations
    # print()

Asuncion
Huanuco
Chiclayo 
El Collao 
Nasca 
Santa
Utcubamba
Huamalies
Barranca 
Zarumilla 
Castrovirreyna
Carabaya 
Pachitea
Lambayeque 
Cajabamba
Gran Chimu 
Sanchez Carrion 
Arequipa
Cañete 
Huancane 
Angaraes
Atalaya 
Cotabambas
Luya
Junin 
Yauyos 
Talara 
La Mar
San Martin 
Urubamba
Coronel Portillo 
San Antonio de Putina 
Celendin
Espinar
Mariscal Luzuriaga
Pacasmayo 
Moyobamba 
Tayacaja
Huaraz
Antonio Raymondi
Julcan 
Rioja 
Datem del Marañon 
Chanchamayo 
Viru 
Sullana 
Lucanas
Canas
Calca
Padre Abad 
Rodriguez de Mendoza
Ferreñafe 
Cangallo
Caylloma
Jaen
Lampa 
Chucuito 
Puno 
Sucre
Mariscal Ramon Castilla 
Purus
Chachapoyas
Concepcion 
Daniel Alcides Carrion 
San Ignacio
Islay
Cajatambo 
Santiago de Chuco 
El Dorado 
San Pablo
Loreto 
Parinacochas
Moho 
Contralmirante Villar 
Pisco 
Melgar 
Vilcas Huaman
Jauja 
General Sanchez Cerro 
Maynas 
Pomabamba
Oyon 
Huaral 
Corongo
Ucayali 
Huanta
Pasco 
Candarave 
San Miguel
Huancabamba 
Bongara
Carlos Fermin Fitzcarrald
Sandia 
Pait